# Yelp Reviews Rating Prediction

Bangwei Zhou (bz2280) Github: bzhouuu<br>
James Jungsuk Lee (jl5241) Github: yjs1210<br>
Ujjwal Peshin (up2138) Github: ujjwal95<br>
Zhongling Jiang (zj2249) Github: jiangzl2016<br>

## A. Abstract

We compare the merits of n different recommendation algorithms side by side.

* Collaborative Filtering (CF - Baseline)
* Collective Matrix Factorization (CMF)
* Content-Based Filtering (CBF)
* Field-aware Factorization Machine (FFM) 
* Deep Learning Model

* Hybrid Approach <br>

[RESULTS]

[CONCLUSION AND RECOMMENDATION]


## B. Business Case and Objective

Our goal through this exercise is to figure out the best methodology for recommending restaurants to our users. This will be largely measured by using RMSE, which measures our forecast’s error relative to the ground truth as well as ranking metrics. Additionally, we want to measure metrics that can help us assess the sanity of our recommendation and experiences of the users. We measure these using coverage and novelty metrics to ensure that there is enough diversity in our recommendations and that our model isn’t resorting to recommending the most popular restaurants as opposed to creating a truly personalized experience. 
 
There are additional considerations such as model computation time, which affects long term overhead of maintenance of the product, and model comprehension. These are important considerations for the business as inefficient solutions will add technology debt and burden to the company’s systems. Also, employing models that are well understood can help the team in getting buy-in from stakeholders. Therefore, these factors will be critically analyzed when assessing the merits of our algorithms. 


## C. Data

We use the publicly available yelp dataset which consists roughly of 7 million reviews from 1.6 million users across 190 thousand businesses in 10 metropolitan areas. We are additionally provided with business and user metadata as well as check-in information, tips and photos provided by the reviewers. 

In [2]:
# !pip install tqdm

In [65]:
import pandas as pd
import tarfile
from tqdm import tqdm, tqdm_notebook, tnrange
import json
import numpy as np
import time
from copy import deepcopy

In [16]:
zf = tarfile.open('yelp_dataset.tar') 
zf.extract('review.json')

In [17]:
line_count = len(open("review.json").readlines())
user_ids, business_ids, stars, dates, texts = [], [], [], [], []
with open("review.json") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        user_ids += [blob["user_id"]]
        business_ids += [blob["business_id"]]
        stars += [blob["stars"]]
        dates += [blob["date"]]
        texts += [blob["text"]]
ratings_ = pd.DataFrame(
    {"user_id": user_ids, "business_id": business_ids, "rating": stars, "date": dates, "text": texts}
)
user_counts = ratings_["user_id"].value_counts()
active_users = user_counts.loc[user_counts >= 5].index.tolist()
ratings_ = ratings_.loc[ratings_.user_id.isin(active_users)]

100%|██████████| 6685900/6685900 [01:04<00:00, 104276.79it/s]


In [18]:
ratings_.head()

,business_id,date,rating,text,user_id
0,ujmEBvifdJM6h6RLv4wQIg,2013-05-07 04:34:36,1.0,Total bill for this horrible service? Over $8G...,hG7b0MtEbXx5QzbzE6C_VA
2,WTqjgwHlXbSFevF32_DJVw,2016-11-09 20:09:03,5.0,I have to say that this office really has it t...,n6-Gk65cPZL6Uz8qRm3NYw
6,3fw2X5bZYeW9xCz_zGhOHg,2016-05-07 01:21:02,3.0,Tracy dessert had a big name in Hong Kong and ...,jlu4CztcSxrKx56ba1a5AQ
7,zvO-PJCpNk4fgAVUnExYAA,2010-10-05 19:12:35,1.0,This place has gone down hill. Clearly they h...,d6xvYpyzcfbF_AZ8vMB7QA
8,b2jN2mm9Wf3RcrZCgfo1cg,2015-01-18 14:04:18,2.0,I was really looking forward to visiting after...,sG_h0dIzTKWa3Q6fmb4u-g


In [19]:
n_users = len(ratings_.user_id.unique())
n_restaurants = len(ratings_.business_id.unique())
print('Unique Users: {0}, unique restaurants: {1}'.format(n_users, n_restaurants))

Unique Users: 286130, unique restaurants: 185723


Load in User attributes set and Restaurant attributes set 

In [20]:
users_ = pd.read_csv('active_users.csv')
business_ = pd.read_csv('businesses.csv')

In [21]:
users_.head()

,user_id,user_name,user_review_count,user_yelping_since,friends,useful_reviews,funny_reviews,cool_reviews,n_fans,years_elite,average_stars
0,l6BmjZMeQD3rDxWUbiAiow,Rashmi,95,2013-10-08 23:11:33,"c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",84,17,25,5,"2015,2016,2017",4.03
1,4XChL029mKr5hydo79Ljxg,Jenna,33,2013-02-21 22:29:06,"kEBTgDvFX754S68FllfCaA, aB2DynOxNOJK9st2ZeGTPg...",48,22,16,4,NaN,3.63
2,bc8C_eETBWL0olvFSJJd0w,David,16,2013-10-04 00:16:10,"4N-HU_T32hLENLntsNKNBg, pSY2vwWLgWfGVAAiKQzMng...",28,8,10,0,NaN,3.71
3,MM4RJAeH6yuaN8oZDSt0RA,Nancy,361,2013-10-23 07:02:50,"mbwrZ-RS76V1HoJ0bF_Geg, g64lOV39xSLRZO0aQQ6DeQ...",1114,279,665,39,"2015,2016,2017,2018",4.08
4,TEtzbpgA2BFBrC0y0sCbfw,Keane,1122,2006-02-15 18:29:35,"RJQTcJVlBsJ3_Yo0JSFQQg, GWt_h78k1CBBkE1NpThGfQ...",13311,19356,15319,696,"2006,2007,2008,2009,2010,2011,2012,2013",4.39


In [22]:
business_.head()

,business_id,business_name,business_address,business_city,business_state,business_latitude,business_longitude,stars,review_counts,is_open,categories
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,33.522143,-112.018481,3.0,5,0,"Golf, Active Life"
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,43.605499,-79.652289,2.5,128,1,"Specialty Food, Restaurants, Dim Sum, Imported..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,35.092564,-80.859132,4.0,170,1,"Sushi Bars, Restaurants, Japanese"
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,33.455613,-112.395596,5.0,3,1,"Insurance, Financial Services"
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,35.190012,-80.887223,4.0,4,1,"Plumbing, Shopping, Local Services, Home Servi..."


## E. Subsampling and Train-Test Split
#### E.1. FIltering, Undersampling and Dataset Size Considerations
The idea behind undersampling is to develop a smaller dataset that is representable, or partially representative of the entire dataset, and expedites the model development cycle. We utilize 20% dataset size generally across the board for fast development iteration and for computation intensive tasks such as hyperparameter tuning. However, it is critical to analyze model performance across dataset sizes since the models’ capability to handle large dataset is an important consideration if it were to be utilized in production. Therefore we investigate two other dataset sizes as well, 50% and 100% of the data size. Whereas 20% of the dataset size is used generally, larger datasets are employed to analyze certain models’ capability to handle large data as well as measure how their performance changes with larger datasets. Finally, we filter our dataset to only active users who have at least 5 reviews since there needs to be at least some data about the user for the model to perform. If a user does not have at least 5 reviews, we will be building out recommendations using cold-start methods.

#### E.2 Train Test Split

Train-Test split is done in a very straightforward way. We take all the users that made through our filters as described above. Then we simply take the last two reviews of the users. The most recent review becomes our test set and the other second most recent review becomes our validation set. 


In [23]:
SAMPLING_RATE = 1/5

In [24]:
# Downsample by users
user_id_unique = ratings_.user_id.unique()
user_id_sample = pd.DataFrame(user_id_unique, columns=['unique_user_id']) \
                    .sample(frac= SAMPLING_RATE, replace=False, random_state=1)

ratings_sample = ratings_.merge(user_id_sample, left_on='user_id', right_on='unique_user_id') \
                    .drop(['unique_user_id'], axis=1)
print(ratings_sample.head())
print(ratings_sample.shape)

              business_id                 date  rating  \
0  WTqjgwHlXbSFevF32_DJVw  2016-11-09 20:09:03     5.0   
1  hk5wpV-_pi5jmDDVPeG8DA  2018-09-14 18:50:19     5.0   
2  30Q5xBagQHmkwp8Q9I1FCg  2018-02-03 23:27:43     5.0   
3  UtWngqS-WloIY_A53W5K-Q  2016-02-18 06:42:16     5.0   
4  dU-Nt1-LjV9mAgFOVcdAJw  2018-08-15 22:14:18     5.0   

                                                text                 user_id  
0  I have to say that this office really has it t...  n6-Gk65cPZL6Uz8qRm3NYw  
1  I highly recommend Arizona Pet Mortuary, David...  n6-Gk65cPZL6Uz8qRm3NYw  
2  First time at this restaurant our server "Ramo...  n6-Gk65cPZL6Uz8qRm3NYw  
3  Such an amazing hospital with friendly staff, ...  n6-Gk65cPZL6Uz8qRm3NYw  
4  Went for my yearly GYN exam and was seen by Lo...  n6-Gk65cPZL6Uz8qRm3NYw  
(918368, 5)


In [25]:
# hold out last review
ratings_user_date = ratings_sample.loc[:, ['user_id', 'date']]
ratings_user_date.date = pd.to_datetime(ratings_user_date.date)
index_holdout = ratings_user_date.groupby(['user_id'], sort=False)['date'].transform(max) == ratings_user_date['date']
ratings_holdout_ = ratings_sample[index_holdout]
ratings_traincv_ = ratings_sample[~index_holdout]

ratings_user_date = ratings_traincv_.loc[:, ['user_id', 'date']]
index_holdout = ratings_user_date.groupby(['user_id'], sort=False)['date'].transform(max) == ratings_user_date['date']
ratings_cv_ = ratings_traincv_[index_holdout]
ratings_train_ = ratings_traincv_[~index_holdout]

# remove the user that has fake reviews 

cv_users_del = set(ratings_cv_.user_id) - set(ratings_train_.user_id)
holdout_users_del = set(ratings_holdout_.user_id) - set(ratings_train_.user_id)
ratings_cv_ = ratings_cv_[~ratings_cv_.user_id.isin(cv_users_del)]
ratings_holdout_ = ratings_holdout_[~ratings_holdout_.user_id.isin(holdout_users_del)]

# ratings_cv_ = ratings_cv_[~ratings_cv_.user_id.isin(['HiT9sg9pvDiEVMFHJYihXg'])]
# ratings_holdout_ = ratings_holdout_[~ratings_holdout_.user_id.isin(['HiT9sg9pvDiEVMFHJYihXg'])]

print('There are {0} rows, {1} columns in training set.'.format(ratings_train_.shape[0], ratings_train_.shape[1]))
print('There are {0} rows, {1} columns in training set.'.format(ratings_cv_.shape[0], ratings_cv_.shape[1]))
print('There are {0} rows, {1} columns in holdout set.'.format(ratings_holdout_.shape[0], ratings_holdout_.shape[1]))

There are 803897 rows, 5 columns in training set.
There are 57229 rows, 5 columns in training set.
There are 57223 rows, 5 columns in holdout set.


In [46]:
# check if we have a enough user sample size (> 50000)
number_of_unique_users = len(ratings_train_.user_id.unique())
print(number_of_unique_users)

57223


## F. Evaluation Metrics

A list of evaluation metrics the team uses are:

#### F.1. Regression Metrics
Regression metrics measure We present four different regression metrics that are measured but we primarily use the RMSE. 
Root Mean Square Error (RMSE) calculates square rooted sum of square residuals of predictions. It measures numerical difference between all ground-truth ratings and actual ratings in test set.
Mean Absolute Error (MAE) calculates sum of absolute residuals. It measures numerical difference between all ground-truth ratings and actual ratings in test set, and more robust to outliers.
R-squared measures what percentage of variance in target that is explained by predictions. The higher the value, the better the predictions.

#### F.2. Ranking Metrics
We first make top 10 recommendations of restaurants for each user and see how relevant the rankings were. First we discuss the metrics then discuss a few caveats to the approach we took to measure them.
Inclusion of Last Review in Top 10 Recommendations: We train our model on the training set and make top 10 recommendations to the users. We then look at the test set and see if the user’s latest review was included in that top 10 recommendations. We calculate the proportion of users who received such recommendations. In other words we have,

$$\frac{1}{n} * \sum_j^n \sum_{i \in j's top 10}^{10}Rel(i)$$

where a recommendation is relevant if it is the latest visited restaurant of the user and n is the total number of users measured. 
Average Ranking of Latest Restaurant: We train our model on the training set and make a prediction on every single business  and user combination. We then measure the average rankings on that prediction of the latest business that the user visited. In other words we have,

$$\frac{1}{n} * \sum_j^n\sum_i^m Rank(i)(1_{i=latest business of j})$$

Where the indicator function is one if the restaurant is the last visited restaurant of user j, n is the total number of users and m is the total number of businesses.
Some caveats of our approach is that instead of making a prediction on the entire business universe, we make predictions on the businesses that are in the same city as the business of the user’s last review. This makes sense since it would be futile to recommend a restaurant in Los Angeles to a user who is in New York City. This also allows us to reduce the computation time of our recommendation, which is another key advantage we want to have when we serve our model to the users. Additionally, we measure the above two metrics on a subsample of users as opposed to all the users to save computation time, and we also only take subsample of ratings that were positive ratings since we want to measure how good our recommendations are. 
#### F.3. Coverage
We measure the coverage of our recommendations by looking at the proportion of our recommendations that are distinct. In other words, we measure, <br>

$$\frac{number of distinct businesses in all recommendations to the subsample} 
{number of all recommendations to the subsample}$$

 
#### F.4. Novelty
We measure the novelty of our recommendations by simply taking the proportion of businesses in our top ten recommendations that the user hasn’t been to. This is crucial since we don’t want our recommendations to be filled with restaurants that the user has already been to. We measure novelty as simply. 

#### F.5. Runtime 
We measure the runtime of the model’s training as well as its prediction time on validation set using Python’s time library. 

#### Metrics on Coverage and Serendipity

First subsample a group of users that we will measure these metrics from

Methodology:
    We sample 5 users from each city where the user made the latest review.
    These cities must have at least 100 unique businesses
    These users must also have made a postive review(above their historical average)to those restaurants.
        1. We recommend 10 restaurants to each user
        2. We see if their latest restaurant makes it into the top 10 list (Ranking Metric)
        3. We see for those 10 x 5 recommendations, how many of them are distinct businesses (Coverage)
        4. We see for those top 10 recommendations, how many of them are restaurants they have not visited (Serendipity)
    
    Additionally, we measure what our ranking was for the latest restaurant that the user visited(Ranking Metric
    


Criteria: 

In [27]:
def process(df):
    df['date']  = pd.to_datetime(df['date'])
    df['week_day'] = df['date'].dt.weekday
    df['month'] = df['date'].dt.month
    df['hour'] = df['date'].dt.hour
    df = df.merge(users_, on = 'user_id')
    df = df.merge(business_, on = 'business_id')
    rename_dict = {'business_longitude': 'longitude', 'business_latitude': 'latitude',
                  'business_state':'state','business_city':'city', 'business_address': 'address'}
    df = df.rename(columns = rename_dict)
    return df

ratings_train = process(ratings_train_.copy())
ratings_holdout = process(ratings_holdout_.copy())
ratings_val = process(ratings_cv_.copy())

In [28]:
ratings_train_final = ratings_train.append(ratings_val)
ratings_entire_df = ratings_train.append(ratings_val).append(ratings_holdout)

In [34]:
ratings_holdout.columns

Index(['business_id', 'date', 'rating', 'text', 'user_id', 'week_day', 'month',
       'hour', 'user_name', 'user_review_count', 'user_yelping_since',
       'friends', 'useful_reviews', 'funny_reviews', 'cool_reviews', 'n_fans',
       'years_elite', 'average_stars', 'business_name', 'address', 'city',
       'state', 'latitude', 'longitude', 'stars', 'review_counts', 'is_open',
       'categories'],
      dtype='object')

In [30]:
unique_city_businesses = ratings_entire_df[['city','business_id']].drop_duplicates()
unique_cities = unique_city_businesses.groupby('city').count()['business_id']
unique_cities = unique_cities[unique_cities > 100]
out = pd.DataFrame()
for city in unique_cities.index:
    tmp = ratings_holdout[(ratings_holdout['city'] ==city) &
                              (ratings_holdout['rating'] >ratings_holdout['average_stars'])]
    if len(tmp['user_id'].unique())>4:
        
        ###this weird sampling technique is to ensure we dont' sample the same user twice in a same city
        five_users = np.random.choice(tmp['user_id'].unique(),5, replace = False)
        row = tmp[tmp['user_id'].isin(five_users)].groupby('user_id', group_keys=False).apply(lambda df: df.sample(1))
        out = out.append(row)

In [31]:
predict_df = out[['user_id','city','state']]
predict_df = predict_df.merge(unique_city_businesses, on = 'city')
predict_df.to_csv('data/metric_sample.csv')

In [32]:
# predict_df['predictions'] = 2.5

In [33]:
def get_all_metrics(predict_df, validation_subsample, ratings_train_final):
    top_10_recs = predict_df.groupby(['user_id','city'])['predictions'].nlargest(10).reset_index()
    out = validation_subsample
    cnt =0
    serendipity = 0
    
    
    for row in out.iterrows():
        row_values = row[1]
        top_10 = predict_df.loc[top_10_recs[top_10_recs['user_id'] == row_values['user_id']].level_2]['business_id']
        ###In top 10
        if row_values['business_id'] in top_10.values:
            cnt+=1
        user_history = ratings_train_final[ratings_train_final['user_id'] == row_values['user_id']]    
        been_there = [i for i in top_10.values if i in  user_history.business_id.values]
        serendipity += 1-len(been_there)/10
    
    top_10 = cnt/len(out)
    serendipity = serendipity/len(out)
    
    predict_df = predict_df.reset_index()
    
    analysis_df = predict_df.merge(top_10_recs, left_on = ['user_id','city','index'], \
                                   right_on = ['user_id','city','level_2'])
    
    coverage = (analysis_df.groupby('city')['business_id'].nunique()/50).values.mean()
    
    predict_df['rankings']=predict_df.groupby(['city','user_id'])['predictions']. \
                                                        rank(method="first",ascending = False)
    running_rankings =0
    for row in out.iterrows():
        row_values = row[1]
        user_recs = predict_df[(predict_df['user_id']==row_values['user_id'])
                            &(predict_df['city']==row_values['city'])
                             & (predict_df['business_id']==row_values['business_id'])
                              ]
        assert len(user_recs)==1
        running_rankings += user_recs['rankings'].sum()

    avg_rank = running_rankings / len(out)
    print(top_10, coverage, serendipity, avg_rank)
    
    return top_10, coverage, serendipity, avg_rank

## F. Methods

A list of models that the team attempts are: 

* Bias Baseline
* Collaborative Filtering Baseline: SVD
* Collective Matrix Factorization (CMF)
* Content-Based Filtering (CBF)
* Field-aware Factorization Machine (FFM) 
* Deep Learning Model

_**Note**_: the team has run algorithms on **20%, 50%, 100%** training data respectively. But for readability purpose, only result on 20% data is displayed in this notebook. For full result, please refer to a result table enclosed in **pdf report**.

### Baseline models

### Baseline 1: Bias Baseline

$\sum_{r_{ui} \in R_{train}} \left(r_{ui} - (\mu + b_u + b_i)\right)^2 +
\lambda \left(b_u^2 + b_i^2 \right)$.

#### Hyperparameter Tuning

In [ ]:
from surprise import SVD
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import GridSearchCV
from surprise import Dataset
from surprise import BaselineOnly

In [197]:
bsl_options = {'method': 'als', 'n_epochs':3}
bias_baseline = BaselineOnly(bsl_options)
algo.fit(train_sr_20)
predictions = algo.test(val_sr_20)
accuracy.rmse(predictions)

RMSE: 1.3492


1.3491711762452891

In [198]:
bsl_options = {'method': 'als', 'n_epochs':5}
bias_baseline = BaselineOnly(bsl_options)
algo.fit(train_sr_20)
predictions = algo.test(val_sr_20)
accuracy.rmse(predictions)

RMSE: 1.3492


1.3491711762452891

In [199]:
bsl_options = {'method': 'als', 'n_epochs':7}
bias_baseline = BaselineOnly(bsl_options)
algo.fit(train_sr_20)
predictions = algo.test(val_sr_20)
accuracy.rmse(predictions)

RMSE: 1.3492


1.3491711762452891

In [200]:
bsl_options = {'method': 'als', 'n_epochs':9}
bias_baseline = BaselineOnly(bsl_options)
algo.fit(train_sr_20)
predictions = algo.test(val_sr_20)
accuracy.rmse(predictions)

RMSE: 1.3492


1.3491711762452891

**Observation**: It seems different hyperparameters all performs the same result; the team just uses default.

#### Evaluation

**Note**: the team has evaluated this and the following algorithms on **20%, 50%, 100%** training data respectively. For readability, only result on 20% data is displayed. For full result, please refer to a result table enclosed.

In [180]:
# 20%
start_time = time.time()
bias_baseline.fit(train_sr_20)
print("--- %s seconds ---" % (time.time() - start_time))

Estimating biases using als...
--- 2.0564420223236084 seconds ---


In [188]:
# 20%
bbase_p = bias_baseline.test(test_sr_20)
start_time = time.time()
bbase_20_df = pd.DataFrame(bbase_p, columns = ['userId','itemId','rating','pred_rating','x'])
accuracy.rmse(bbase_p)
print('R^2 (with 20% data): ', r2_score(bbase_20_df.rating , bbase_20_df.pred_rating))
print('MAE (with 20% data): ', mean_absolute_error(bbase_20_df.rating, bbase_20_df.pred_rating))
print("--- %s seconds ---" % (time.time() - start_time))

RMSE: 1.3545
R^2 (with 20% data):  0.19799189125456051
MAE (with 20% data):  1.127068744947832
--- 0.08617496490478516 seconds ---


In [204]:
algo = BaselineOnly()
eval_before_50 = eval_20.build_full_trainset()
eval_sr_20 = eval_before_20.build_testset()
algo.fit(train_sr_20)
eval_pred_20 = algo.test(eval_sr_20)
#accuracy.rmse(predictions_50)
baseline_20 = pd.DataFrame(eval_pred_20, columns = ['userId','itemId','rating','pred_rating','x'])

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


In [205]:
top_10, coverage, serendipity, avg_rank = get_all_metrics(predict_df_20, out_20, ratings_train_final_20)

0.10714285714285714 0.503095238095238 0.9697619047619042 528.3333333333334


### Baseline 2: Collaborative Filtering via SVD

Matrix factorization is a class of collaborative filtering algorithms. The general idea behind matrix factorization is that there can exist a lower dimensional latent space of features in which users and items can be represented such that the interaction between them can be obtained by simply dot producing the corresponding dense vectors in that space. In short, it decomposes a m*n user-item interaction matrix into two m*k and k*n matrices, sharing a joint latent vector space, where m represents the number of users, and n represents the number of items. In terms of its outcome, we are likely to observe that close users in terms of preferences as well as close items in terms of characteristics can have close representations in the latent space.

The mathematical overview is as follows:
Given a n*m matrix, such that . X is the user matrix where rows represent the n users and Y is the item matrix where rows represent the m items. We want to search for the dot product of matrices X and Y that best approximate the existing interactions; i.e., we want to find X and Y that minimize the “rating reconstruction error”:

$$ (X,Y) = argmin_{X,Y} \sum_{(i,j) \in E} [(X_i)(Y_j)^T − M_{ij}]^2$$

Adding a regularization term, we can also get:

$$(X,Y) = argmin_{X,Y} ½ \sum_{(i,j) \in E} [(X_i)(Y_j)^T − M_{ij}]^2 + \lambda/2(\sum_{i,k}(X_{ik})^2 + \sum_{j,k}(Y_{jk})^2)$$

In general, we obtain the matrices X and Y following a gradient descent optimization process. And once the matrices are obtained, we can predict the ratings simply by multiplying the user vector by any item vector.

In this Yelp Rating Challenge, we used the python surprise package to implement MF. The MF algorithm there uses the SVD approach, which is essentially 

$$ P_{m * n} = U_{m * m} \sum_{m * n} V_{n * n}$$

There, the prediction is
 $$\hat(r_{ui}) = \mu + b_u + b_i + (q_i)^T p_u $$
 
and the regularized squared error that needs to be minimized is 

$$\sum_{r_{ui} \in R_{train}} (r_{ui} − \hat(r_{ui}))^2 + \lambda(b^2_{i} + b^2_{u} + ||q_i||^2 + ||p_u||^2)$$

As the way the package is designed, we tuned on n_epochs, lr_all and leg_all to get an optimal hyperparameter set, where n_epochs is the number of iterations of the SGD (stochastic gradient descent) procedure, lr_all is the learning rate for all parameters, and reg_all is the regularization term for all parameters.  


In [11]:
#!pip install cmfrec
# !pip install scikit-surprise

In [12]:
from cmfrec import CMF
from surprise import KNNBasic

import matplotlib.pyplot as plt
import json
from tqdm import tqdm

/home/jzljohn18/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [23]:
def process(df):
    df['date']  = pd.to_datetime(df['date'])
    df['week_day'] = df['date'].dt.weekday
    df['month'] = df['date'].dt.month
    df['hour'] = df['date'].dt.hour
    df = df.merge(users_, on = 'user_id')
    df = df.merge(business_, on = 'business_id')
    rename_dict = {'business_longitude': 'longitude', 'business_latitude': 'latitude',
              'business_state':'state','business_city':'city', 'business_address': 'address'}
    df = df.rename(columns = rename_dict)
    return df
ratings_train = process(ratings_train_.copy())
ratings_test = process(ratings_holdout_.copy())
ratings_val = process(ratings_cv_.copy())

In [24]:
ratings_test = ratings_test.loc[ratings_test.business_id.isin(ratings_train.business_id)]
ratings_val = ratings_val.loc[ratings_val.business_id.isin(ratings_train.business_id)]

In [25]:
trainset = ratings_train.loc[:,['user_id', 'business_id', 'rating']]
trainset.columns = ['userID', 'itemID','rating']
valset = ratings_val.loc[:, ['user_id', 'business_id', 'rating']]
valset.columns = ['userID', 'itemID','rating']
testset = ratings_holdout.loc[:, ['user_id', 'business_id', 'rating']]
testset.columns = ['userID', 'itemID','rating']

In [26]:
reader = Reader(rating_scale = (0.0, 5.0))
train_data = Dataset.load_from_df(trainset[['userID','itemID','rating']], reader)
val_data = Dataset.load_from_df(valset[['userID','itemID','rating']], reader)
test_data = Dataset.load_from_df(testset[['userID','itemID','rating']], reader)

train_sr = train_data.build_full_trainset()
val_sr_before = val_data.build_full_trainset()
val_sr = val_sr_before.build_testset()
test_sr_before = test_data.build_full_trainset()
test_sr = test_sr_before.build_testset()

#### Hyperparameter Tuning

In [119]:
RMSE_tune = {}
n_epochs = [5, 7, 10]  # the number of iteration of the SGD procedure
lr_all = [0.002, 0.003, 0.005] # the learning rate for all parameters
reg_all =  [0.4, 0.5, 0.6] # the regularization term for all parameters

for n in n_epochs:
    for l in lr_all:
        for r in reg_all:
            algo = SVD(n_epochs = n, lr_all = l, reg_all = r)
            algo.fit(train_sr)
            predictions = algo.test(val_sr)
            RMSE_tune[n,l,r] = accuracy.rmse(predictions)

RMSE: 1.4171
RMSE: 1.4166
RMSE: 1.4172
RMSE: 1.4012
RMSE: 1.4023
RMSE: 1.4032
RMSE: 1.3794
RMSE: 1.3805
RMSE: 1.3820
RMSE: 1.4048
RMSE: 1.4047
RMSE: 1.4054
RMSE: 1.3888
RMSE: 1.3888
RMSE: 1.3902
RMSE: 1.3643
RMSE: 1.3664
RMSE: 1.3681
RMSE: 1.3897
RMSE: 1.3906
RMSE: 1.3914
RMSE: 1.3721
RMSE: 1.3733
RMSE: 1.3749
RMSE: 1.3496
RMSE: 1.3513
RMSE: 1.3529


In [127]:
import operator
min(RMSE_tune.items(), key=operator.itemgetter(1))[0]

(10, 0.005, 0.4)

**Observation**: The best combination is when n_epochs = 10, lr_all = 0.005, reg_all = 0.4, and the RMSE score is 1.3496

#### Evaluation

In [30]:
# train and test on the optimal parameter
start_time = time.time()
algo_real = SVD(n_epochs = 10, lr_all = 0.005, reg_all = 0.4)
algo_real.fit(train_sr)
predictions = algo_real.test(test_sr)

In [31]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 33.18766689300537 seconds ---


In [32]:
accuracy.rmse(predictions)

RMSE: 1.3985


1.398543135413844

In [33]:
accuracy.mae(predictions)

MAE:  1.1848


1.1847820428584857

In [35]:
predict_df_20 = pd.read_csv('data/metric_sample.csv', index_col=0)
predict_df_20.columns

Index(['user_id', 'city', 'state', 'business_id'], dtype='object')

In [52]:
# To evaluate coverage and serendipity metrics, use evaluation set created earlier.
predict_df_20 = pd.read_csv('data/metric_sample.csv', index_col=0)
predict_df_20['predictions'] = 2.5 # fill in this value temporally
eval_20 = Dataset.load_from_df(predict_df_20[['user_id','business_id','predictions']], reader)

In [53]:
eval_before_20 = eval_20.build_full_trainset()
eval_sr_20 = eval_before_20.build_testset()
eval_pred_20 = algo_real.test(eval_sr_20)

baseline_20 = pd.DataFrame(eval_pred_20, columns = ['userId','itemId','rating','pred_rating','x'])
predict_df_20['predictions'] = baseline_20.pred_rating

In [56]:
top_10, coverage, serendipity, avg_rank = get_all_metrics(predict_df_20, out, ratings_train_final)

0.10930232558139535 0.4932558139534884 0.9762790697674414 518.7511627906977


### Models

### (i) Collective Matrix Factorization (CMF)

Collective Matrix Factorization method decomposes two matrices $X$ and $Y$ into three matrices $U$, $V$, and $Z$ such that $X \approx f(UV^T)$ and $ Y \approx f(VZ^T)$, where f is either the identity or sigmoid function. This allows us to include additional features of our users or items that might help optimize the result of personalization. 

**Approach 1: CMF on User, Business and State Average Rating**<br>
In the first approach, we calculated state average rating as an additional feature on the items (i.e., the restaurants). The idea is as follows: our goal is to predict the last rating for each active user. To bring closer our prediction to the actual rating, we felt that state average rating can help. In other words, "eaters" should have similar pickiness when it comes to foods and restaurants in the same state. They know the best about the expectation they should have for restaurants around them as the locals, and we wanted to take into account of this aspect. In particular, we didn't do the average calculation by summing up from the business dataset, because that will be somewhat "leaking" the information, since we are holding out the last rating of our active users. Thus, we calculated the ratings from the training ratings by user by state.

For example, (and in fact from our EDA, though with limited data), people in CA seem to be less critical with foods and restaurants as they tend to have a higher state average rating (perhaps, it already takes away a lot of energy to drive around to the restaurants as CA is such a massive land, so they just complain less), whereas people in NY really have opinions and take very personally with what they intake as their state average rating tends to be lower (perhaps, and it seems reasonable, that NY is known as the food hub, so people definitely will have a higher expectation for foods). In other words, we are trying to make up this very "subjective" assumption about the eating behaviors of the yelp users (that they really know what they are eating) and we will test the performance of our model to see how it works.

In [58]:
users = users_
businesses = business_

def process(df):
    df['date']  = pd.to_datetime(df['date'])
    df['week_day'] = df['date'].dt.weekday
    df['month'] = df['date'].dt.month
    df['hour'] = df['date'].dt.hour
    df = df.merge(users, on = 'user_id')
    df = df.merge(businesses, on = 'business_id')
    rename_dict = {'business_longitude': 'longitude', 'business_latitude': 'latitude',
                  'business_state':'state','business_city':'city', 'business_address': 'address'}
    df = df.rename(columns = rename_dict)
    return df

In [59]:
ratings_holdout_100 = ratings_holdout_
ratings_train_100 = ratings_train_
ratings_val_100 = ratings_cv_

ratings_val_100 = process(ratings_val_100.copy())
ratings_train_100 = process(ratings_train_100.copy())
ratings_holdout_100 = process(ratings_holdout_100.copy())

ratings_test_100 = ratings_holdout_100.loc[ratings_holdout_100.business_id.isin(ratings_train_100.business_id)]
ratings_val_100 = ratings_val_100.loc[ratings_val_100.business_id.isin(ratings_train_100.business_id)]

trainset_100 = ratings_train_100.loc[:,['user_id', 'business_id', 'rating']]
trainset_100.columns = ['userID', 'itemID','rating']
valset_100 = ratings_val_100.loc[:, ['user_id', 'business_id', 'rating']]
valset_100.columns = ['userID', 'itemID','rating']
testset_100 = ratings_holdout_100.loc[:, ['user_id', 'business_id', 'rating']]
testset_100.columns = ['userID', 'itemID','rating']

ratings_train_final_100 = ratings_train_100.append(ratings_val_100)
ratings_entire_df_100 = ratings_train_100.append(ratings_val_100).append(ratings_holdout_100)


### (ii) Content Based Model

**Approach 1: Predict rating via user reviews**<br>

Implemented according to **CF-MCM (User-Item based)** approach in this following paper: https://arxiv.org/pdf/1607.00024.pdf

The idea of this model is to predict the rating of a review given by user _U_ to restaurant _R_ by comparing the review to the ones written by other users who have also reviewed restaurant _R_. The predicted rating is calculated by taking a weighted average of other users' ratings, where the weight is calculated based on similarity of reviews. Here is the detail: 

<img src="image/formula.png" width="700">

Two users'ratings are compared in the following manner: first aggregate all reviews user _u_ and user _a_ have written; then bucket them according to ratings given (from r = 1.0 to 5.0); finally find the max of all 5 pairs of similarity. 

<img src="image/text_comparison.png" width="700">

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
import string
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jzljohn18/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [35]:
ratings_train = ratings_train_.copy()
ratings_cv = ratings_cv_.copy()
ratings_holdout = ratings_holdout_.copy()

In [36]:
def clean_text(text):
    # substitute some irregular context 
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"\s{2,}", " ", text)
    # remove numbers
    text = re.sub(r"\d+", "", text)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    # remove punctuations from each word
    table = str.maketrans('','', string.punctuation)
    text = [w.translate(table) for w in text]
    
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    # Replace slang terms
    # Word stemming
    
#     porter = PorterStemmer()
#     text = [porter.stem(word) for word in text]
    
    text = " ".join(text)
    
    return text

ratings_train['text'] = ratings_train['text'].apply(clean_text)

In [37]:
ratings_train.head()

,business_id,date,rating,text,user_id
0,WTqjgwHlXbSFevF32_DJVw,2016-11-09 20:09:03,5.0,say office really together organized friendly ...,n6-Gk65cPZL6Uz8qRm3NYw
2,30Q5xBagQHmkwp8Q9I1FCg,2018-02-03 23:27:43,5.0,first time restaurant server ramone asked firs...,n6-Gk65cPZL6Uz8qRm3NYw
3,UtWngqS-WloIY_A53W5K-Q,2016-02-18 06:42:16,5.0,amazing hospital friendly staff nurses doctors...,n6-Gk65cPZL6Uz8qRm3NYw
5,76aGN20BrGWvAlYfPGVv_A,2016-04-05 15:25:11,5.0,let start saying grew father mechanic retired ...,n6-Gk65cPZL6Uz8qRm3NYw
6,FUcwrXBb_ljg3LgTqt6F2g,2018-08-15 22:07:13,5.0,love pharmacy compounded prescription cats hyp...,n6-Gk65cPZL6Uz8qRm3NYw


In [38]:
# concatnate all reviews for each business for each rating value
ratings_train_sub = ratings_train[['user_id', 'business_id', 'rating', 'text']]
# ratings_train_sub['ct'] = 0
userid_df = ratings_train_sub.groupby(['user_id', 'rating']).agg({'text': ' '.join, 'business_id': 'count'})
# retain a copy of indexed dataset
businessid_userid_df = ratings_train_sub.set_index(['user_id', 'business_id', 'rating'])

## Learn TF-IDF vector representation for each concatenated review
# which will be used for similarity calculation
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, analyzer='word', \
                             stop_words='english', max_features=50)
userid_tfidf_fit = userid_vectorizer.fit(userid_df['text'])

In [39]:
# Algorithm implementation 

def predict_approach_1(selected_uid, selected_bid): 
    # cosine similarity
    def text_similarity(v1, v2):
        return np.dot(v1, v2.T).toarray()[0][0]
    
    pred_rating = -1

    users_reviews = userid_df

    selected_business = businessid_userid_df.xs(selected_bid, level='business_id').reset_index()
    users_who_review_business = selected_business.user_id

    user_reviews_userID = users_reviews.xs(selected_uid, level='user_id')

    w_ui_u_vec = []
    for ui in users_who_review_business:
        user_reviews_ui = users_reviews.xs(ui, level='user_id')
        rating_range = [1.0, 2.0, 3.0, 4.0, 5.0]
        valid_rating_range_1 = user_reviews_userID.reset_index()['rating'].tolist()
        valid_rating_range_2 = user_reviews_ui.reset_index()['rating'].tolist()
        w_ui_u = -100

        for r in rating_range:
            if (r not in valid_rating_range_1) or (r not in valid_rating_range_2):
                continue
            else:
                user_reviews_r = user_reviews_userID.loc[r, 'text']
                ui_reviews_r = user_reviews_ui.loc[r, 'text']
                user_reviews_r_vec = userid_tfidf_fit.transform([user_reviews_r])
                ui_reviews_r_vec = userid_tfidf_fit.transform([ui_reviews_r])
                # similarity_r = text_similarity(user_reviews_r, ui_reviews_r)
                similarity_r = text_similarity(user_reviews_r_vec, ui_reviews_r_vec)
                if similarity_r > w_ui_u:
                    w_ui_u = similarity_r
        w_ui_u_vec.append(w_ui_u)

    tmp = user_reviews_userID.reset_index()
    r_u_bar = sum(tmp.rating * tmp.business_id) / sum(tmp.business_id) # business_id is number of ratings 
                                                                        # that has correpsonding value 
    if sum(w_ui_u_vec) == 0:
        pred_rating = r_u_bar + sum(w_ui_u_vec \
                                * (selected_business.rating - np.mean(selected_business.rating))) \
                                * 1.0 / sum(w_ui_u_vec)
    else:
        pred_rating = r_u_bar
    return pred_rating

In [42]:
tmp_df = ratings_cv[ratings_cv['user_id'].isin(ratings_train.user_id)].reset_index()
ratings_cv_filtered = tmp_df[tmp_df['business_id'].isin(ratings_train.business_id)]

tmp_df = ratings_holdout[ratings_holdout['user_id'].isin(ratings_train.user_id)].reset_index()
ratings_holdout_filtered = tmp_df[tmp_df['business_id'].isin(ratings_train.business_id)]

In [45]:
approach_1_start = time.time()

predict_df_1 = ratings_cv_filtered.loc[:, ['user_id', 'business_id', 'rating']]
predict_df_1['pred_rating'] = -1

for (u, b) in tqdm(predict_df_1.iterrows(), position=0, leave=True):
    try:
        pred = predict_approach_1(b.user_id, b.business_id)
        predict_df_1.loc[u, 'pred_rating'] = pred 
    except AttributeError:
        if sum(predict_df_1.pred_rating == -1) == 0:
            print('Finished')
        else:
            print('Error: Not Finised')
            print('Currently at u = {}'.format(u))
approach_1_duration = (time.time() - approach_1_start) * 1.0 / 60 
print('The approach 1 takes {} minutes to run.'.format(approach_1_duration))

1172it [07:08,  3.02it/s]/home/jzljohn18/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in double_scalars
53225it [4:48:43,  3.07it/s]

The approach 1 takes 288.7285438974698 minutes to run.


In [47]:
predict_df_1.head()

,user_id,business_id,rating,pred_rating
0,n6-Gk65cPZL6Uz8qRm3NYw,dU-Nt1-LjV9mAgFOVcdAJw,5.0,3.857143
1,d6xvYpyzcfbF_AZ8vMB7QA,d10IxZPirVJlOSpdRZJczA,1.0,2.800000
2,sG_h0dIzTKWa3Q6fmb4u-g,BxJAl-LoiSiIHonoGzVu7Q,2.0,3.400000
3,FIk4lQQu1eTe2EpzQ4xhBA,gcouHCQrswvakJ6xSEKtFQ,4.0,4.124476
4,TpyOT5E16YASd7EWjLQlrw,sYSlKRCWmVeQr1hA6-WUzw,4.0,3.725275


In [74]:
predict_df_1.shape

(53225, 4)

In [55]:
# Remove rows without predictions
predict_df_1_NA_removed = predict_df_1.loc[~predict_df_1.pred_rating.isnull(), :]
predict_df_1_NA_removed.shape

(53162, 4)

In [56]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [57]:
mse, mae, r2 = mean_squared_error(predict_df_1_NA_removed.rating, predict_df_1_NA_removed.pred_rating), \
                mean_absolute_error(predict_df_1_NA_removed.rating, predict_df_1_NA_removed.pred_rating), \
                r2_score(predict_df_1_NA_removed.rating, predict_df_1_NA_removed.pred_rating)
print('MSE: {0:.4f}, MAE: {1:.4f}, R2: {2:.4f}'.format(mse, mae, r2))

MSE: 2.1025, MAE: 1.1308, R2: 0.0218


In [70]:
# predict_df_target = pd.read_csv('data/metric_sample.csv', index_col=0)
# # predict_df_target['predictions'] = 2.5 # fill in this value temporally
# # predict_df_1.loc[predict_df_1.user_id == predict_df_target.user_id && \
# #                  predict_df_1.business_id == predict_df_target.business_id, :]

# predict_df_target_combined = pd.merge(predict_df_target, predict_df_1,  how='left', left_on=['user_id','business_id'], \
#          right_on = ['user_id','business_id'])
# # top_10, coverage, serendipity, avg_rank = get_all_metrics(predict_df_target, out, ratings_train_final)

**Approach 2: (Implementation please see Appendix)**

Build User Profile & Business Profile based on business categories information. As the algorithm takes too long to run, the code is attached in appendix. The idea is

### (iii) Field-Aware Factorization Machine

In [9]:
# !pip install CMake
# Installation guide https://linxid.github.io/2018/05/10/Xlearn/
# !pip install xlearn
# !pip install regex

In [37]:
ratings_train_.head()

,business_id,date,rating,text,user_id
0,WTqjgwHlXbSFevF32_DJVw,2016-11-09 20:09:03,5.0,I have to say that this office really has it t...,n6-Gk65cPZL6Uz8qRm3NYw
2,30Q5xBagQHmkwp8Q9I1FCg,2018-02-03 23:27:43,5.0,"First time at this restaurant our server ""Ramo...",n6-Gk65cPZL6Uz8qRm3NYw
3,UtWngqS-WloIY_A53W5K-Q,2016-02-18 06:42:16,5.0,"Such an amazing hospital with friendly staff, ...",n6-Gk65cPZL6Uz8qRm3NYw
5,76aGN20BrGWvAlYfPGVv_A,2016-04-05 15:25:11,5.0,Let me start by saying that I grew up with a f...,n6-Gk65cPZL6Uz8qRm3NYw
6,FUcwrXBb_ljg3LgTqt6F2g,2018-08-15 22:07:13,5.0,"Love this pharmacy, they compounded a prescrip...",n6-Gk65cPZL6Uz8qRm3NYw


In [62]:
import xlearn as xl
import networkx as nx
pd.options.mode.chained_assignment = None
import regex as re
import math

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [41]:
ratings_train.columns

Index(['business_id', 'date', 'rating', 'text', 'user_id', 'week_day', 'month',
       'hour', 'user_name', 'user_review_count', 'user_yelping_since',
       'friends', 'useful_reviews', 'funny_reviews', 'cool_reviews', 'n_fans',
       'years_elite', 'average_stars', 'business_name', 'business_address',
       'business_city', 'business_state', 'business_latitude',
       'business_longitude', 'stars', 'review_counts', 'is_open', 'categories',
       'state_avg', 'text_cluster', 'graph_cluster'],
      dtype='object')

In [50]:
def process(df):
    df['date']  = pd.to_datetime(df['date'])
    df['week_day'] = df['date'].dt.weekday
    df['month'] = df['date'].dt.month
    df['hour'] = df['date'].dt.hour
    df = df.merge(users_, on = 'user_id')
    df = df.merge(business_, on = 'business_id')
    rename_dict = {'business_longitude': 'longitude', 'business_latitude': 'latitude',
                  'business_state':'state','business_city':'city', 'business_address': 'address'}
    df = df.rename(columns = rename_dict)
    return df

ratings_train = process(ratings_train_.copy())
ratings_holdout = process(ratings_holdout_.copy())
ratings_val = process(ratings_cv_.copy())

In [51]:
def _convert_to_ffm(path, df, type, target, numerics, categories, features, encoder):
    # Flagging categorical and numerical fields
    print('convert_to_ffm - START')
    for x in numerics:
        if(x not in encoder['catdict']):
            print(f'UPDATING CATDICT: numeric field - {x}')
            encoder['catdict'][x] = 0
    for x in categories:
        if(x not in encoder['catdict']):
            print(f'UPDATING CATDICT: categorical field - {x}')
            encoder['catdict'][x] = 1

    nrows = df.shape[0]
    with open(path + str(type) + "_ffm.txt", "w") as text_file:

        # Looping over rows to convert each row to libffm format
        for n, r in enumerate(range(nrows)):
            datastring = ""
            datarow = df.iloc[r].to_dict()
            datastring += str(int(datarow[target]))  # Set Target Variable here

            # For numerical fields, we are creating a dummy field here
            for i, x in enumerate(encoder['catdict'].keys()):
                if(encoder['catdict'][x] == 0):
                    # Not adding numerical values that are nan
                    if math.isnan(datarow[x]) is not True:
                        datastring = datastring + " "+str(i)+":" + str(i)+":" + str(datarow[x])
                else:

                    # For a new field appearing in a training example
                    if(x not in encoder['catcodes']):
                        print(f'UPDATING CATCODES: categorical field - {x}')
                        encoder['catcodes'][x] = {}
                        encoder['currentcode'] += 1
                        print(f'UPDATING CATCODES: categorical value for field {x} - {datarow[x]}')
                        encoder['catcodes'][x][datarow[x]] = encoder['currentcode']  # encoding the feature

                    # For already encoded fields
                    elif(datarow[x] not in encoder['catcodes'][x]):
                        encoder['currentcode'] += 1
                        print(f'UPDATING CATCODES: categorical value for field {x} - {datarow[x]}')
                        encoder['catcodes'][x][datarow[x]] = encoder['currentcode']  # encoding the feature

                    code = encoder['catcodes'][x][datarow[x]]
                    datastring = datastring + " "+str(i)+":" + str(int(code))+":1"

            datastring += '\n'
            text_file.write(datastring)

    return encoder

#### Feature Generation 

In the following steps, different sets of features will be generated as input to FFM.


(i) Create user review **text embeddings** based on training data only

In [52]:
rating_texts = ratings_train.groupby(['user_id'])['text'].apply(lambda x: ','.join(x)).reset_index()
rating_texts['text'] = rating_texts['text'].str.lower()
rating_texts['text'] = rating_texts['text'].str.replace(r"[^a-zA-Z ]+", " ").str.strip()

vectorizer = TfidfVectorizer(lowercase=True, stop_words = 'english', strip_accents = 'ascii')
vectorizer.fit(rating_texts['text'])
vector = vectorizer.transform(rating_texts['text'])
tsv = TruncatedSVD(n_components=50)
tsv.fit(vector)
transformed_tsv = tsv.transform(vector)

'''
wcss=[]
for i in range(77, 100):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(transformed_tsv)
    wcss.append(kmeans.inertia_)
plt.plot(range(2, 100), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()
'''

kmeans = KMeans(n_clusters=110, init='k-means++', max_iter=300, n_init=10, random_state=0)
kmeans.fit(transformed_tsv)
text_cluster = kmeans.predict(transformed_tsv)

rating_texts.loc[:,'text_cluster'] = text_cluster
rating_texts_features = rating_texts[['user_id','text_cluster']]
ratings_train= ratings_train.merge(rating_texts_features[['user_id','text_cluster']], on ='user_id', how = 'left')
ratings_holdout = ratings_holdout.merge(rating_texts_features[['user_id','text_cluster']], on ='user_id', how = 'left')
ratings_val = ratings_val.merge(rating_texts_features[['user_id','text_cluster']], on ='user_id', how = 'left')

In [54]:
ratings_train.head(1)

,business_id,date,rating,text,user_id,week_day,month,hour,user_name,user_review_count,...,city,state,latitude,longitude,stars,review_counts,is_open,categories,state_avg,text_cluster
0,WTqjgwHlXbSFevF32_DJVw,2016-11-09 20:09:03,5.0,I have to say that this office really has it t...,n6-Gk65cPZL6Uz8qRm3NYw,2,11,20,Wilhelmina,10,...,Chandler,AZ,33.259702,-111.790203,3.5,39,1,"Health & Medical, Cosmetic Dentists, Orthodont...",3.707185,82


(ii) Create **graph** features based on user friends attribute.

In [55]:
train_users = ratings_train[['user_id','friends']].drop_duplicates()
train_users_dict = train_users.set_index('user_id').T.to_dict('list')

g = nx.Graph()
g.add_nodes_from(train_users_dict.keys())

for k, v in train_users_dict.items():
    for i in v[0].split(','):
        g.add_edge(k,i.strip())    

sub_graphs = nx.connected_component_subgraphs(g)

sgs =[]
for i, sg in enumerate(sub_graphs):
    sgs += [sg]
fin_sgs = []
for i in sgs:
    if len(i.nodes()) >=5:
        fin_sgs +=[i]

graph_user_ids, graph_ids = [],[]
num_id = 0
for graph in fin_sgs:
    for node in graph.nodes():
        graph_user_ids += [node]
        graph_ids += [num_id]
    num_id += 1
social_graphs = pd.DataFrame(
    {"user_id": graph_user_ids, "graph_cluster": graph_ids}
)

ratings_train= ratings_train.merge(social_graphs, on ='user_id', how = 'left')
ratings_holdout = ratings_holdout.merge(social_graphs, on ='user_id', how = 'left')
ratings_val = ratings_val.merge(social_graphs, on ='user_id', how = 'left')


(iii) Create **Location** Features: business longitude and latitude

In [56]:
X = ratings_train[['longitude','latitude']]

''' 
EXPLORATORY ANALYSIS TO DETERMINE NUMBER OF CLUSTERS. DON'T RUN
HERE USING THE EBLOW METHOD WE CHOOSE NCLUSTERs OF 10 

X = ratings_train[['longitude','latitude']]
wcss = []
for i in range(2, 100):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
plt.plot(range(2, 50), wcss[0:48])
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()
'''

kmeans = KMeans(n_clusters=10, init='k-means++', max_iter=300, n_init=10, random_state=0)
kmeans.fit(X)

ratings_train.loc[:,'location_cluster'] = kmeans.predict(ratings_train[['longitude','latitude']])
ratings_holdout.loc[:,'location_cluster'] = kmeans.predict(ratings_holdout[['longitude','latitude']])
ratings_val.loc[:,'location_cluster'] = kmeans.predict(ratings_val[['longitude','latitude']])

ratings_train.text_cluster.fillna('999', inplace=True)
ratings_holdout.text_cluster.fillna('999', inplace=True)
ratings_val.text_cluster.fillna('999', inplace=True)

ratings_train.location_cluster.fillna('999', inplace=True)
ratings_holdout.location_cluster.fillna('999', inplace=True)
ratings_val.location_cluster.fillna('999', inplace=True)

ratings_train.graph_cluster.fillna('999', inplace=True)
ratings_holdout.graph_cluster.fillna('999', inplace=True)
ratings_val.graph_cluster.fillna('999', inplace=True)

In [57]:
class FFMModel:

    def __init__(self, train, test, config, suffix = None):
        self.train_df = train
        self.test_df = test
        self.model = xl.create_ffm()
        self.suffix = suffix
        self.config = config
        self.preds = None 
          
    def __configure(self):
        destination = self.config['destination']
        label = self.config['label']
        numerical_columns  = self.config['numerical_columns']
        categorical_columns  = self.config['categorical_columns']
        all_columns  = numerical_columns + categorical_columns
              
        encoder = {"currentcode": len(self.config['numerical_columns']),
           "catdict": {},
           "catcodes": {}}
        
        encoder = _convert_to_ffm(destination, self.train_df , 'train', label, numerical_columns, \
                                  categorical_columns, all_columns, encoder)
        encoder = _convert_to_ffm(destination, self.test_df , 'test', label, numerical_columns, \
                                  categorical_columns, all_columns, encoder)
        
    def train(self, params = None):
        encoder = self.__configure()
        self.model.setTrain(self.config['destination']+'train_ffm.txt')
        self.model.setValidate(self.config['destination']+'test_ffm.txt')
        self.model.setTest(self.config['destination']+'test_ffm.txt')
        
        if not params:
            params = {'task': 'reg',
                     'lr': 0.2,
                     'lambda': 0.002,
                     'metric': 'auc'}
        
        self.model.fit(params, self.config['model_destination']+self.config['model_name']+'.out')

    def evaluate_on_val(self):
        self.model.predict(self.config['model_destination'] + self.config['model_name']+'.out', \
                           self.config['output_destination']+'predictions.txt')
        preds = pd.read_csv(self.config['output_destination']+'predictions.txt', sep=" ", header=None)
        preds.columns = ['prediction']
        preds.prediction = np.clip(preds.prediction,0.0,5.0)
        self.preds = preds
        return preds            
    
    def get_regression_metrics(self):
        if self.preds is None:
            self.evaluate_on_val()
        
        predictions = self.evaluate_on_val()
        test_df = self.test_df.copy()
        test_df['preds']  = np.clip(predictions.values,0.0,5.0)
        
        r2 = r2_score(test_df['rating'], test_df['preds'])
        mse = mean_squared_error(test_df['rating'], test_df['preds'])
        mae = mean_absolute_error(test_df['rating'], test_df['preds'])
        
        print('R2: ', r2, ' MSE: ',mse, ' MAE: ', mae)
        return r2,mse,mae

#### Model Fitting and Evaluation

Now run FFM on different combinations of features and calculate R2, mean squared error (MSE) and mean absolute error (MAE) on them.

#### Metrics with just userid and business id 

In [64]:
baseline_config = {   'destination': 'data/',
    'categorical_columns':['business_id','user_id'],
    'numerical_columns' : [],
    'label' : 'rating',
    'model_destination' : 'trained_models/',
    'model_name' : 'baseline',
    'output_destination': 'output/'
}

In [65]:
baseline_FFM = FFMModel(ratings_train, ratings_val, baseline_config)

In [66]:
%%capture
baseline_FFM.train()

In [67]:
r2, mse, mae = baseline_FFM.get_regression_metrics()

R2:  0.1873866119640475  MSE:  1.7830047851238464  MAE:  1.0886505079082125


#### Metrics with Baseline + Business Location

In [68]:
user_business_config = {   'destination': 'data/',
    'categorical_columns':['business_id','user_id','city','state'],
    'numerical_columns' : [],
    'label' : 'rating',
    'model_destination' : 'trained_models/',
    'model_name' : 'user_business',
    'output_destination': 'output/'
}

In [69]:
user_business_FFM = FFMModel(ratings_train, ratings_val, user_business_config)

In [70]:
%%capture
user_business_FFM.train()

In [71]:
r2, mse, mae = user_business_FFM.get_regression_metrics()

R2:  0.1964585848489021  MSE:  1.763099414005971  MAE:  1.0615091977315227


#### Metrics with Baseline + Location Cluster (has best MSE)

In [58]:
location_config = {   'destination': 'data/',
    'categorical_columns':['business_id','user_id','location_cluster','city','state'],
    'numerical_columns' : [],
    'label' : 'rating',
    'model_destination' : 'trained_models/',
    'model_name' : 'location',
    'output_destination': 'output/'
}

In [59]:
location_FFM = FFMModel(ratings_train, ratings_val, location_config)

In [60]:
%%capture
location_FFM.train()

In [63]:
r2, mse, mae = location_FFM.get_regression_metrics()

R2:  0.19744277000305088  MSE:  1.760939953104722  MAE:  1.0666155431063107


#### Metrics with User and Business Information + Text Cluster

In [72]:
text_config = {   'destination': 'data/',
    'categorical_columns':['business_id','user_id','text_cluster','city','state'],
    'numerical_columns' : [],
    'label' : 'rating',
    'model_destination' : 'trained_models/',
    'model_name' : 'text',
    'output_destination': 'output/'
}
text_FFM = FFMModel(ratings_train, ratings_val, text_config)

In [73]:
%%capture
text_FFM.train()

In [74]:
r2, mse, mae = text_FFM.get_regression_metrics()

R2:  0.18948085767652612  MSE:  1.778409672390595  MAE:  1.064447769604502


#### Metrics with User and Business Information + Graph Cluster

In [75]:
graph_config = {   'destination': 'data/',
    'categorical_columns':['business_id','user_id','graph_cluster','city','state'],
    'numerical_columns' : [],
    'label' : 'rating',
    'model_destination' : 'trained_models/',
    'model_name' : 'graph',
    'output_destination': 'output/'
}
graph_FFM = FFMModel(ratings_train, ratings_val, graph_config)

In [76]:
%%capture
graph_FFM.train()

In [77]:
r2, mse, mae = graph_FFM.get_regression_metrics()

R2:  0.19572440525321522  MSE:  1.7647103224053686  MAE:  1.072474083328964


#### Metrics All clusters

In [78]:
all_config = {   'destination': 'data/',
    'categorical_columns':['business_id','user_id','graph_cluster','location_cluster','text_cluster'],
    'numerical_columns' : [],
    'label' : 'rating',
    'model_destination' : 'trained_models/',
    'model_name' : 'all_clusters',
    'output_destination': 'output/'
}
all_FFM = FFMModel(ratings_train, ratings_val, all_config)

In [79]:
%%capture
all_FFM.train()

In [80]:
r2, mse, mae = all_FFM.get_regression_metrics()

R2:  0.1862330235045122  MSE:  1.7855359441887817  MAE:  1.065033877918174


**Observation**: The baseline with location cluster performs best as it has lowest MSE. Therefore, this model will be used. The following analysis is done on this model.

#### Run Time Analysis

FFM runs in linear time which leads to fast runtime. Our model takes just two minutes to run.

In [81]:
%%capture
import time
start_time = time.time()
location_FFM = FFMModel(ratings_train, ratings_val, location_config)
location_FFM.train()

In [82]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 206.21558594703674 seconds ---


In [83]:
%%capture
start_time = time.time()
location_FFM.get_regression_metrics()

In [84]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.16828489303588867 seconds ---


#### Coverage and Serendipity metrics 

In [99]:
predictions = location_FFM.evaluate_on_val()
predict_df['predictions'] = predictions['prediction']

In [100]:
top_10, coverage, serendipity, avg_rank = get_all_metrics(predict_df, out, ratings_train_final)

0.011627906976744186 0.8576470588235293 0.9990697674418605 72.29767441860466


### (iv) Deep Learning models

In [58]:
!pip install -U deepctr[cpu]

    100% |████████████████████████████████| 81kB 2.6MB/s ta 0:00:011
    100% |████████████████████████████████| 61kB 10.2MB/s ta 0:00:01
    100% |████████████████████████████████| 2.9MB 418kB/s eta 0:00:01
Requirement already up-to-date: tensorflow!=1.7.*,!=1.8.*,>=1.4.0; extra == "cpu" in /home/jzljohn18/anaconda3/lib/python3.6/site-packages (from deepctr[cpu])
    100% |████████████████████████████████| 133kB 8.9MB/s eta 0:00:01
    100% |████████████████████████████████| 163kB 7.1MB/s eta 0:00:01
Requirement already up-to-date: chardet<3.1.0,>=3.0.2 in /home/jzljohn18/anaconda3/lib/python3.6/site-packages (from requests->deepctr[cpu])
    100% |████████████████████████████████| 61kB 10.3MB/s ta 0:00:01
    100% |████████████████████████████████| 20.0MB 60kB/s  eta 0:00:01
Requirement already up-to-date: keras-preprocessing>=1.0.5 in /home/jzljohn18/anaconda3/lib/python3.6/site-packages (from tensorflow!=1.7.*,!=1.8.*,>=1.4.0; extra == "cpu"->deepctr[cpu])
Requirement already up-to

In [59]:
import os
import shutil
import sys

from scipy import sparse

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sn
sn.set()

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr.models import DeepFM, CCPM, FNN, PNN, WDL, MLR, NFM, AFM, DCN, \
                            DIN, DIEN, DSIN, xDeepFM, AutoInt, NFFM, FGCNN, FiBiNET
from deepctr.inputs import SparseFeat,get_feature_names

In [65]:
training = ratings_train_.copy()
validation = ratings_cv_.copy()
test = ratings_holdout_.copy()

In [66]:
# convert object to datetime
# training.date = pd.to_datetime(training.date)
# validation.date = pd.to_datetime(validation.date)
# test.date = pd.to_datetime(test.date)

# # find hour from datetime
# training['hour'] = training.date.dt.hour
# validation['hour'] = validation.date.dt.hour
# test['hour'] = test.date.dt.hour

test = test.loc[test.business_id.isin(training.business_id)]
validation = validation.loc[validation.business_id.isin(training.business_id)]

# map each user_id, business_id to an index
# user_mapping = {}
# for n,i in enumerate(training.user_id.unique()):
#     user_mapping[i] = n

# business_mapping = {}
# for n,i in enumerate(training.business_id.unique()):
#     business_mapping[i] = n

# # for training
# training['user_id'] = training['user_id'].map(user_mapping)
# training['business_id'] = training['business_id'].map(business_mapping)
# # for validation
# validation['user_id'] = validation['user_id'].map(user_mapping)
# validation['business_id'] = validation['business_id'].map(business_mapping)
# # for test
# test['user_id'] = test['user_id'].map(user_mapping)
# test['business_id'] = test['business_id'].map(business_mapping)

test.head()

,business_id,date,rating,text,user_id
45,qdCwzhJ5Yo_Sdm_bYDIfOQ,2011-09-11 06:09:33,2.0,I found Kathy's from yelp. I love to support ...,d6xvYpyzcfbF_AZ8vMB7QA
74,XS1Zx6GzjtKPKmhDuVw5Jg,2017-06-19 22:55:06,3.0,I had the Saison infused with grapefruit which...,sG_h0dIzTKWa3Q6fmb4u-g
472,jLxeBgWhLRbII2ACkgH1Sg,2018-09-30 18:00:41,4.0,First time for me to come inside at least! Hav...,FIk4lQQu1eTe2EpzQ4xhBA
854,U_yacPCk8HgE1ywATmQUrg,2018-10-13 00:10:59,5.0,Ordered for lunch with a few colleagues throug...,TpyOT5E16YASd7EWjLQlrw
899,O-b5osM0NO4f31dp6_DatQ,2014-08-01 01:55:23,3.0,"I can only comment on their macarons, which I'...",_N7Ndn29bpll_961oPeEfw


In [67]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
sparse_features = ["user_id", "business_id"]#, "hour"]
target = ['rating']
for feat in sparse_features:
    lbe = LabelEncoder()
    training[feat] = lbe.fit_transform(training[feat])
    print("hi!")
    validation[feat] = lbe.transform(validation[feat])
    print("hii!")
    test[feat] = lbe.transform(test[feat])
    print("hiii!")

# 2.count #unique features for each sparse field
fixlen_feature_columns = [SparseFeat(feat, training[feat].nunique(), \
                                     embedding_dim = 8) for feat in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
print("hiiii!")
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

train_model_input = {name:training[name].values for name in feature_names}
valid_model_input = {name:validation[name].values for name in feature_names}
print("hiiiii!")
test_model_input = {name:test[name].values for name in feature_names}

# 4.Define Model,train,predict and evaluate
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression', \
            dnn_hidden_units = (128, 128), dnn_dropout = 0.3, l2_reg_embedding=1e-05, \
            l2_reg_dnn=1e-05, l2_reg_linear=1e-05)

model.compile("adam", "mse", metrics=['mse'], )

history = model.fit(train_model_input, training[target].values, batch_size=256, \
                    epochs=10, verbose=2, validation_data= (valid_model_input, validation[target].values))

hi!


KeyboardInterrupt: 

In [ ]:
pred_ans = model.predict(test_model_input, batch_size=256)

print("test MSE", round(mean_squared_error(
        test[target].values, pred_ans), 4))

pred_ans

(test[target].values<2).sum(), (test[target].values<3).sum(), (test[target].values<4).sum()

(pred_ans<2).sum(),(pred_ans<3).sum(), (pred_ans<4).sum() 

### Appendix

In [7]:
# Select restaurants in training set
print(business.shape)
train_business_id = np.unique(ratings_train.business_id)
business = business.loc[business.business_id.isin(train_business_id), :]
print(business.shape)

### Business Profile

tmp_series = pd.Series(business.category.str.split(', ').tolist(), index=business.business_id)
business_df = pd.get_dummies(tmp_series.apply(pd.Series).stack()).sum(level=0).reset_index()

business_df.head()

business_df_normalized = business_df.set_index('business_id') \
                            .div(business_df.set_index('business_id').sum(axis=1)**0.5, axis=0)

business_df_normalized.sort_values('business_id', inplace=True)

business_df_normalized.head()

### User Profile

# check if all restaurants that users have rated exist in restuarant profile
print('The number of all rated restaurants: {}'.format(len(np.unique(ratings_train[['business_id']]))))
print('The number of resturants existing profile: {}'.format(len(np.unique(business_df.business_id))))

# **Caveat**: Even for a subsample (ratings_train is a subsample of full ratings dataset), there are restaurants rated by users but not **do not exist** in business profile dataset. This needs to be handled when building recommender system. 

# Build uer profile
users_id = np.unique(ratings_train.user_id)
users_df = pd.DataFrame(columns = business_df_normalized.columns)

approach_2_start = time.time()
# create a for loop to create profile vector for each user
for i in tqdm(range(len(users_id))):
    selected_user_id = users_id[i]
    # select business that user has rated
    selected_rating = ratings_train.loc[ratings_train.user_id == selected_user_id, ['business_id', 'rating']]. \
                                        set_index('business_id')
    ind = selected_rating.index
    # To calculate user profile vector, 
    # multiply weight (normalized frequency) by ratings across all rated business and take mean of the result
    try: 
        # only need to extract rated restaurants because the rest of term will be 
        w = business_df_normalized.loc[ind, :]
    except KeyError:
        business_id_existed_profile = business_df.business_id
        filtered_ind = pd.Index([j for j in ind if j in business_id_existed_profile.tolist()])    
        w = business_df_normalized.loc[filtered_ind, :]
    except:
        print('Some other errors occured during iteration i = {}'.format(i))
    if (w.shape[0] == 0):
        users_df.loc[selected_user_id] = 0
    else:
        profile_vec = w.mul(selected_rating.loc[:, 'rating'], axis=0).mean(axis=0)
        users_df.loc[selected_user_id] = profile_vec
        
approach_2_duration = (time.time() - approach_2_start) * 1.0 / 60 
print('The approach 2 takes {} minutes to run.'.format(approach_2_duration)) 

users_df.shape

### CHECK
# IDF vector  
DF = business_df_normalized.sum(axis=0)
IDF = (business_df.shape[0]/DF).apply(np.log) #log inverse of DF
# The dot product of business profile vector and IDF vector gives
# weighted score of each business
IDF_business_df_normalized = business_df_normalized.mul(IDF.values)
IDF_business_df_normalized.head()

SyntaxError: invalid syntax (<ipython-input-7-323e9717061a>, line 26)